## Import libraries

In [1]:
# reload(sys)
import pandas as pd
import numpy as np
import os
import random
from pdb import set_trace as bp
import matplotlib.pyplot as plt
from collections import defaultdict
import pickle
import sys 
from sklearn.utils import shuffle
from dataGenerator import DataGenerator
from models import QuoraModel
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import text_to_word_sequence
from utils import plotGraph,plotLengthHistogram,plotHist,labelPlot
from utils import createTiles, plotHistory
from modelTrainer import ModelTrainer

Using TensorFlow backend.


In [2]:
## Define parameters
timeSlice = 2
questionLen = 25
wordEmbeddDim = 100
batchSize  = 16
nEpochs = 10

In [3]:
dataGenerator = DataGenerator(batchSize = batchSize, 
                              questionLen = questionLen,
                              wordEmbeddDim = wordEmbeddDim)
nData = dataGenerator.nTrainData

Data Generator succesfully initialized :) :) :) 


In [22]:
# This returns a tensor
from keras.optimizers import Nadam
net = QuoraModel(questionLen = questionLen,
                   wordEmbeddDim = wordEmbeddDim)
model = net.getModel1(load_weight = False)
nadam = Nadam(lr=1e-7, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(optimizer=nadam,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 2, 25, 100)        0         
_________________________________________________________________
batch_norm_1 (BatchNormaliza (None, 2, 25, 100)        400       
_________________________________________________________________
lstm_1 (TimeDistributed)     (None, 2, 25, 500)        1202000   
_________________________________________________________________
batch_norm_2 (BatchNormaliza (None, 2, 25, 500)        2000      
_________________________________________________________________
lstm_2 (TimeDistributed)     (None, 2, 250)            751000    
_________________________________________________________________
flatten (Flatten)            (None, 500)               0         
_________________________________________________________________
batch_norm_3 (BatchNormaliza (None, 500)               2000      
__________

In [23]:
trainer = ModelTrainer(modelName = 'model1', model = model)

In [7]:
# trainer.plotTrainHist()

TypeError: createTiles() takes at most 4 arguments (5 given)

# train Network

## train Model 1

In [25]:
trainHist = [[], []]
validHist = [[], []]
iterSize = 100
trainX, trainY = dataGenerator.getTrainBatch(2)
nIter = dataGenerator.nTrainData/(batchSize)
for i in range(nEpochs):
    for j in range(nIter):
#         trainX, trainY = dataGenerator.getTrainBatch()
        model.fit( trainX, trainY, verbose = False )
        if( j%50 == 0 ):
#             trainX, trainY = dataGenerator.getTrainBatch(batchSize = 10*batchSize)
            validX, validY = dataGenerator.getValidBatch(batchSize = 10*batchSize)
            trainLoss, trainAcc = model.evaluate( trainX, trainY, verbose = False )
            validLoss, validAcc = model.evaluate(validX, validY, verbose = False )
            trainer.addTrainDetails(trainLoss, trainAcc, validLoss, validAcc, compareWithBest = True)
            print( "Epoch:{0}\t Iter: {1}\t TrainLoss {2:.4f}\t ValidLos: {3:.4f} \
            TrainAcc: {4:.4f}\t ValidAcc: {5:.4f}"
                   .format(i, j+1, trainLoss, validLoss, trainAcc, validAcc))

Epoch:0	 Iter: 1	 TrainLoss 0.6612	 ValidLos: 0.6816             TrainAcc: 1.0000	 ValidAcc: 0.6875
Best loss: 0.66 	 Best Acc: 0.71
Weights saved at 	 ./modelWeights/model1_2017_12_03_13:42_loss_0.6626_acc_0.713.h5py
Epoch:0	 Iter: 51	 TrainLoss 0.6243	 ValidLos: 0.6626             TrainAcc: 1.0000	 ValidAcc: 0.7125
Best loss: 0.66 	 Best Acc: 0.63
Weights saved at 	 ./modelWeights/model1_2017_12_03_13:42_loss_0.6611_acc_0.631.h5py
Epoch:0	 Iter: 101	 TrainLoss 0.5750	 ValidLos: 0.6611             TrainAcc: 1.0000	 ValidAcc: 0.6312
Best loss: 0.65 	 Best Acc: 0.64
Weights saved at 	 ./modelWeights/model1_2017_12_03_13:42_loss_0.6469_acc_0.644.h5py
Epoch:0	 Iter: 151	 TrainLoss 0.5082	 ValidLos: 0.6469             TrainAcc: 1.0000	 ValidAcc: 0.6438
Epoch:0	 Iter: 201	 TrainLoss 0.4599	 ValidLos: 0.7286             TrainAcc: 1.0000	 ValidAcc: 0.5062
Epoch:0	 Iter: 251	 TrainLoss 0.3748	 ValidLos: 0.7372             TrainAcc: 1.0000	 ValidAcc: 0.5437
Epoch:0	 Iter: 301	 TrainLoss 0.3151	

KeyboardInterrupt: 

## train Model 2

In [ ]:
# trainHist = [[], []]
# validHist = [[], []]
# iterSize = 100
# bestLoss = 100
# weightFilePath = "./1234"
# nIter = dataGenerator.nTrainData/(batchSize)
# for i in range(nEpochs):
#     for j in range(nIter):
#         trainX, trainY = dataGenerator.getTrainBatch()
#         model.fit( trainX, trainY, verbose = False )
#         if( j%iterSize == 0 ):
#             trainX, trainY = dataGenerator.getTrainBatch(batchSize = 5*batchSize)
#             validX, validY = dataGenerator.getValidBatch(batchSize = 5*batchSize)
#             trainLoss, trainAcc = model.evaluate( trainX, trainY, verbose = False )
#             validLoss, validAcc = model.evaluate(validX, validY, verbose = False )
#             if validLoss < bestLoss:
#                 bestLoss = validLoss
#                 print("Best loss:{0:.3f}\t BessAcc: {1:.3f} \t Saving weights ".format(validLoss, validAcc))
#                 if os.path.isfile(weightFilePath):
#                     os.remove(weightFilePath)
#                 weightFilePath = 'model/weights_loss_{0:.4f}_acc_{1:.3f}.h5py'.format(validLoss, validAcc)
#                 model.save_weights(weightFilePath)
#             print( "Epoch:{0}\t Iter: {1}\t TrainLoss {2:.4f}\t ValidLos: {3:.4f} \
#             TrainAcc: {4:.4f}\t ValidAcc: {5:.4f}"
#                    .format(i, j+1, trainLoss, validLoss, trainAcc, validAcc))
#             trainHist[0].append(trainLoss)
#             trainHist[1].append(trainAcc)
#             validHist[0].append(validLoss)
#             validHist[1].append(validAcc)
# plotHistory(trainHist, validHist)